<a href="https://colab.research.google.com/github/ashishdasari148/Recommender-Systems-using-Collaborative-Filtering/blob/master/Netflix.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import requests
# Please replace the url with a new api call from "https://www.kaggle.com/netflix-inc/netflix-prize-data" since the API call has timeout.
url = '<use your api key here>'
r = requests.get(url, allow_redirects=True)
dir_name = 'drive/My Drive/Colab Notebooks/Netflix'
open(os.path.join(dir_name, 'data' + "." + 'zip'), 'wb').write(r.content)
import zipfile
with zipfile.ZipFile('data.zip', 'r') as zip_ref:
    zip_ref.extractall(os.path.join(dir_name, 'data'))

In [1]:
import pandas as pd
import re
import os
import time
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
dir_name = 'drive/My Drive/Colab Notebooks/Netflix/data'

In [3]:
# Converting the txt files into csv files
files = ['combined_data_1.txt','combined_data_2.txt','combined_data_3.txt','combined_data_4.txt']
for j in files:
    file1 = open(os.path.join(dir_name, j),"r+")
    print(j + " opened...")
    file2 = open(os.path.join(dir_name, j + '.csv'),"a")
    for i in file1.readlines():
        temp = re.match("(\d*)[:]", i)
        if(temp):
            movienumber = temp.group(1)
        else:
            file2.write(str(movienumber)+","+i)
    file1.close()
    file2.close() 
    print(j + " closed...")

combined_data_1.txt opened...
combined_data_1.txt closed...
combined_data_2.txt opened...
combined_data_2.txt closed...
combined_data_3.txt opened...
combined_data_3.txt closed...
combined_data_4.txt opened...
combined_data_4.txt closed...


In [4]:
movie_titles = pd.read_csv(os.path.join(dir_name, 'movie_titles.csv'), encoding = 'ISO-8859-1', header = None, names=['Movie', 'Year of Release', 'Title'], usecols=[0, 1, 2])
movie_titles

,Movie,Year of Release,Title
0,1,2003.0,Dinosaur Planet
1,2,2004.0,Isle of Man TT 2004 Review
2,3,1997.0,Character
3,4,1994.0,Paula Abdul's Get Up & Dance
4,5,2004.0,The Rise and Fall of ECW
...,...,...,...
17765,17766,2002.0,Where the Wild Things Are and Other Maurice Se...
17766,17767,2004.0,Fidel Castro: American Experience
17767,17768,2000.0,Epoch
17768,17769,2003.0,The Company


In [5]:
## Merge rating data with movie data

# df_raw = pd.read_csv(os.path.join(dir_name, 'combined_data_1.txt.csv'), header=None, names=['Movie', 'User', 'Rating', 'Date'], usecols=[0, 1, 2, 3])
# df_raw = df_raw.merge(movie_titles, on = ['Movie'],how = 'inner')
# df_raw

In [6]:
## extracting additional features from date component

# def add_datepart(df, fldname, drop=True):
#     fld = df[fldname]
#     if not np.issubdtype(fld.dtype, np.datetime64):
#         df[fldname] = fld = pd.to_datetime(fld, infer_datetime_format=True)
#     targ_pre = re.sub('[Dd]ate$', '', fldname)
#     for n in ('Year', 'Month', 'Week', 'Day', 'Dayofweek', 'Dayofyear',
#             'Is_month_end', 'Is_month_start', 'Is_quarter_end', 'Is_quarter_start', 'Is_year_end', 'Is_year_start'):
#         df[targ_pre+n] = getattr(fld.dt,n.lower())
#     # df[targ_pre+'Elapsed'] = fld.astype(np.int64) // 10**9
#     if drop: df.drop(fldname, axis=1, inplace=True)
# add_datepart(df_raw,'Date')

In [7]:
## appending all the csv files of rating data into a single dataframe
dfs = []
for i in ['combined_data_1.txt.csv','combined_data_2.txt.csv','combined_data_3.txt.csv','combined_data_4.txt.csv']:
    df_raw = pd.read_csv(os.path.join(dir_name, 'combined_data_1.txt.csv'), header=None, names=['Movie', 'User', 'Rating', 'Date'], usecols=[0, 1, 2, 3])
    df_raw = df_raw[['Movie','User','Rating']]
    dfs.append(df_raw)
df_raw = pd.concat(dfs, ignore_index=True)

In [ ]:
# saving the merged file to drive for further use.
df_raw.to_csv(path_or_buf=os.path.join(dir_name, 'data.txt.csv'),index=False)

In [8]:
# loading the saved csv into dataframe
df_raw = pd.read_csv(os.path.join(dir_name, 'data.txt.csv'))
df_raw

,Movie,User,Rating
0,1,1488844,3
1,1,822109,5
2,1,885013,4
3,1,30878,4
4,1,823519,3
...,...,...,...
96215051,4499,2591364,2
96215052,4499,1791000,2
96215053,4499,512536,5
96215054,4499,988963,3


In [14]:
# spliting the dataset into training_dataset(80%) and validation_dataset(20%) 

def is_val(x, y):
    return x % 5 == 0
def is_train(x, y):
    return not is_val(x, y)
recover = lambda x,y: y

dataset = tf.data.Dataset.from_tensor_slices((df_raw['Movie'].values,df_raw['User'].values, df_raw['Rating'].values)).shuffle(10, reshuffle_each_iteration=False)
train_dataset = dataset.enumerate().filter(is_train).map(recover).shuffle(len(df_raw)//200).batch(len(df_raw)//5000)
validation_dataset = dataset.enumerate().filter(is_val).map(recover).shuffle(len(df_raw)//200).batch(len(df_raw)//5000)
del dataset
for x, y, z in train_dataset.take(1):
    pass
x,y,z

(<tf.Tensor: shape=(19243,), dtype=int64, numpy=array([172,  44,  28, ..., 166, 108, 118])>,
 <tf.Tensor: shape=(19243,), dtype=int64, numpy=array([2109208, 2581508, 2322592, ..., 1542006, 1691661,   69065])>,
 <tf.Tensor: shape=(19243,), dtype=int64, numpy=array([3, 5, 4, ..., 4, 3, 4])>)

In [10]:
class RecommenderModelDot(tf.keras.Model):
    def __init__(self, no_of_movies, no_of_users, size_of_embeddings=100):
        super(RecommenderModelDot, self).__init__()
        self.size_of_embeddings = size_of_embeddings
        self.bias = tf.random.normal([1], 0, 1, tf.float32)
        self.embeddings_for_movies = tf.keras.layers.Embedding(no_of_movies, size_of_embeddings, embeddings_regularizer=tf.keras.regularizers.L2(l2=0.00001))
        self.embeddings_for_users = tf.keras.layers.Embedding(no_of_users, size_of_embeddings,embeddings_regularizer=tf.keras.regularizers.L2(l2=0.00001))
    def call(self, inputs_movies, inputs_users):
        x = self.embeddings_for_movies(inputs_movies)
        y = self.embeddings_for_users(inputs_users)
        biasx, x = tf.split(x, [1, self.size_of_embeddings-1], 1)
        biasy, y = tf.split(y, [1, self.size_of_embeddings-1], 1)
        return tf.squeeze(tf.math.add_n([self.bias + biasx , biasy , tf.keras.backend.batch_dot(x,y)]))
model_1 = RecommenderModelDot(17770+1, 2649429+1, 100)
model_1(x,y)
model_1.summary()

Model: "recommender_model_dot"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        multiple                  1777100   
_________________________________________________________________
embedding_1 (Embedding)      multiple                  264943000 
Total params: 266,720,100
Trainable params: 266,720,100
Non-trainable params: 0
_________________________________________________________________


In [11]:
class RecommenderModelNeuMF(tf.keras.Model):
    def __init__(self, no_of_movies, no_of_users, size_of_embeddings=100):
        super(RecommenderModelNeuMF, self).__init__()
        self.size_of_embeddings = size_of_embeddings
        self.j = size_of_embeddings//2
        self.DotWeights = tf.random.normal([self.size_of_embeddings-self.j], 0, 1, tf.float32)
        self.embeddings_for_movies = tf.keras.layers.Embedding(no_of_movies, size_of_embeddings, embeddings_regularizer=tf.keras.regularizers.L2(l2=0.00001))
        self.embeddings_for_users = tf.keras.layers.Embedding(no_of_users, size_of_embeddings,embeddings_regularizer=tf.keras.regularizers.L2(l2=0.00001))
        self.hidden_layer_1 = tf.keras.layers.Dense(100, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.L2(l2=0.00001))
        self.hidden_layer_2 = tf.keras.layers.Dense(25, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.L2(l2=0.00001))
        self.output_layer = tf.keras.layers.Dense(1, activation=tf.nn.relu, kernel_regularizer=tf.keras.regularizers.L2(l2=0.00001))
    def call(self, inputs_movies, inputs_users):
        x = self.embeddings_for_movies(inputs_movies)
        y = self.embeddings_for_users(inputs_users)
        j = self.size_of_embeddings//2
        xMLP, xGMF = tf.split(x, [self.j, self.size_of_embeddings-self.j], 1)
        yMLP, yGMF = tf.split(y, [self.j, self.size_of_embeddings-self.j], 1)
        return tf.math.add(tf.math.reduce_sum(tf.math.multiply(yGMF, tf.math.multiply(self.DotWeights, xGMF)), axis=1), 
                          tf.squeeze(self.output_layer(self.hidden_layer_2(self.hidden_layer_1(tf.concat([xMLP,yMLP], axis=1))))))
model_2 = RecommenderModelNeuMF(17770+1, 2649429+1, 100)
model_2(x,y)
model_2.summary()

Model: "recommender_model_neu_mf"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_2 (Embedding)      multiple                  1777100   
_________________________________________________________________
embedding_3 (Embedding)      multiple                  264943000 
_________________________________________________________________
dense (Dense)                multiple                  10100     
_________________________________________________________________
dense_1 (Dense)              multiple                  2525      
_________________________________________________________________
dense_2 (Dense)              multiple                  26        
Total params: 266,732,751
Trainable params: 266,732,751
Non-trainable params: 0
_________________________________________________________________


In [12]:
loss_object = tf.keras.losses.mean_squared_error
def loss(model, x, y, z, training):
    y_ = model(x, y, training=training)
    return loss_object(y_true=z, y_pred=y_)
def grad(model, x, y, targets):
    with tf.GradientTape() as tape:
        loss_value = loss(model, x, y, targets, training=True)
    return loss_value, tape.gradient(loss_value, model.trainable_variables)
optimizer = tf.keras.optimizers.Adam(learning_rate=0.01)

In [13]:
## check if the loss function, gradient and optimization step are working as intended

# for x, y, z in dataset.take(10):
#     loss_value, grads = grad(model_2, x, y, z)
#     optimizer.apply_gradients(zip(grads, model_2.trainable_variables))
#     print("Step: {},         Loss: {}".format(optimizer.iterations.numpy(),loss(model_2, x, y, z, training=True).numpy()))

In [ ]:
# training both the models on the given data
train_loss_results_Dot = []
train_loss_results_MLP = []
validation_loss_results_Dot = []
validation_loss_results_MLP = []
num_epochs = 10
for epoch in range(num_epochs):
    for model in [model_1, model_2]:
        start = time.time()
        epoch_loss_avg_train = tf.keras.metrics.Mean()
        epoch_loss_avg_validation = tf.keras.metrics.Mean()
        progbar = tf.keras.utils.Progbar(1002)
        num=0
        for x, y, z in train_dataset:
            num=num+1
            progbar.update(num)
            loss_value, grads = grad(model, x, y, z)
            optimizer.apply_gradients(zip(grads, model.trainable_variables))
            # Track progress
            epoch_loss_avg_train.update_state(loss_value)  # Add current batch loss
        for x, y, z in validation_dataset:
            num=num+1
            progbar.update(num)
            epoch_loss_avg_validation.update_state(loss(model, x, y, z, training=False))
        # End epoch
        if model == model_1:
            train_loss_results_Dot.append(epoch_loss_avg_train.result().numpy())
            validation_loss_results_Dot.append(epoch_loss_avg_validation.result().numpy())
        else:
            train_loss_results_MLP.append(epoch_loss_avg_train.result().numpy())
            validation_loss_results_MLP.append(epoch_loss_avg_validation.result().numpy())
        end = time.time()
        print("{} Epoch {:03d}: Loss: {:.3f}, Validation Loss: {:.3f}, Time Taken: {}".format(model, epoch, epoch_loss_avg_train.result(), 
                                                                                              epoch_loss_avg_validation.result(), end-start))

In [ ]:
f = plt.figure(figsize=(20,8))
plt.subplot(1, 2, 1)
train_dot, = plt.plot(range(len(train_loss_results_Dot)),train_loss_results_Dot, label='Dot')
train_mlp, = plt.plot(range(len(train_loss_results_MLP)),train_loss_results_MLP, label='MLP')
plt.title('training Losses')
plt.legend(handles=[train_dot,train_mlp]);
plt.subplot(1, 2, 2)
val_dot, = plt.plot(range(len(validation_loss_results_Dot)),validation_loss_results_Dot, label='Dot')
val_mlp, = plt.plot(range(len(validation_loss_results_MLP)),validation_loss_results_MLP, label='MLP')
plt.title('Validation Losses')
plt.legend(handles=[val_dot,val_mlp]);